
**Feito por:**
- Felipe Coelho Silva
- Luiz Felipe Santana dos Santos
- Matheus Inoue
- Anderson Baraldo Junior
- Rômulo Madureira
- Leonardo Bernardi de Avila

## Apresentação do Case

Temos um case de identificação de dígitos, utilizando a base [MNIST](http://yann.lecun.com/exdb/mnist/). 
Neste case devemos aplicar métodos de classificação para, dada a imagem de um dígito manuscrito, identificá-lo.
Para isso, foram implementadas diversas abordagens, dentre as quais destacam-se 3: SVM, Árvore de Decisão e Naive-Bayes.
Um dos motivos para a escolha destas técnicas é por apresentarem características bem distintas entre si (em relação a eficiência, interpretabilidade, etc).
Dado este contexto, o __AHP__ (*Analytic Hierarchy Process*) se mostra como uma técnica adequada para a avaliação dos métodos por considerar os diferentes pesos das características de cada algoritmo, balizando uma melhor decisão.

Em particular, foram analisadas as seguintes características:
- **Complexidade de Implementação:** O quão complicado é implementar a técnica.
- **Tempo de Treinamento:** Tempo necessário para ajustar os parâmetros em um dataset de 60.000 entradas.
- **Tempo de Predição:** Tempo necessário para classificar uma amostra de 10.000 entradas.
- **Interpretabilidade:** Facilidade de explicar as relações causais para obter a decisão tomada pelo algoritmo.
- **Acurácia:** Medida de acerto ([número de classificações corretas]/[total de predições])

Seguindo a lógica do AHP, definimos os graus dos critérios, seguindo a seguir.

### Matriz dos Critérios:

| Matriz dos Critérios |Compl. Implementação     | Tempo de Treinamento | Tempo Pred. | Interpretabilidade | Acurácia        |
|--------------------------|----------------------|-------------|--------------------|----------|-------|
| **Compl. Implementação** | 1                    | 0.5         | 0.333              | 0.666    | 0.333 |
| **Tempo de Treinamento** | 2                    | 1           | 0.666              | 1.333    | 0.666 |
| **Tempo Pred.**          | 3                    | 1.5         | 1                  | 2        | 1     |
| **Interpretabilidade**   | 1.5                  | 0.75        | 0.5                | 1        | 0.5   |
| **Acurácia**             | 3                    | 1.5         | 1                  | 2        | 1     |

### Matrizes dos Graus:

| Compl. Implementação | SVM | Naive-Bayes | Arvore-decisao |
|----------------------|-----|-------------|----------------|
| **SVM**              | 1   | 0.333       | 0.5            |
| **Naive-Bayes**      | 3   | 1           | 1.5            |
| **Arvore-decisao**   | 2   | 0.666       | 1              |



| **Tempo de Treinamento** | SVM | Naive-Bayes | Arvore-decisao |
|--------------------------|-----|-------------|----------------|
| **SVM**                  | 1   | 0.1666      | 0.25           |
| **Naive-Bayes**          | 6   | 1           | 1              |
| **Arvore-decisao**       | 4   | 1           | 1              |



| **Tempo de Predição** | SVM   | Naive-Bayes | Arvore-decisao |
|-----------------------|-------|-------------|----------------|
| **SVM**               | 1     | 1           | 1.5            |
| **Naive-Bayes**       | 1     | 1           | 0.666          |
| **Arvore-decisao**    | 0.666 | 1.5         | 1              |


| **Interpretabilidade** | SVM | Naive-Bayes | Arvore-decisao |
|------------------------|-----|-------------|----------------|
| **SVM**                | 1   | 0.333       | 0.5            |
| **Naive-Bayes**        | 3   | 1           | 1.5            |
| **Arvore-decisao**     | 2   | 0.666       | 1              |



| **Acurácia**       | SVM   | Naive-Bayes | Arvore-decisao |
|--------------------|-------|-------------|----------------|
| **SVM**            | 1     | 1.5         | 1.5            |
| **Naive-Bayes**    | 0.666 | 1           | 1              |
| **Arvore-decisao** | 0.666 | 1           | 1              |

# Código

A idéia inicial foi desenvolver algo que mesmo um especialista que não tenha conhecimento em programação possa utilizar. Assim, optou-se por uma interface bem interativa, mas sem apresentar o código ao usuário.

Ao rodar as células (__Ctrl+[enter]__), o usuário é apresentado com uma série de perguntas quanto ao modelo que este deseja utilizar (número de critérios, alternativas, nomes (*labels*) destes etc.) e alguns avisos são mostrados dependendo da qualidade de seus *inputs* (de forma mais importante, há um aviso para percentual de inconsistência maior que 10%).

Outro ponto importante é a possibilidade de utilizar três métodos diferentes para o cálculo dos autovetores, a saber: `np.linalg.eig` (método do `numpy`, que utiliza as bibliotecas baseadas no `BLAS` (*Basic Linear Algebra Subprograms*)), iterativo e o método dos valores normalizados (ambos baseados no artigo de Oliveira, C. A. e Belderrain, M. C. N. (2008).

In [0]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Clique aqui para mostrar o codigo."></form>''')

In [0]:
# Importação das bibliotecas utilizadas
import numpy as np

In [0]:
def AHP_graus(m, method=1, prints=False):
    '''
    Função para encontrar o maior autovalor e seu autovetor correspondente
    
    Parametros
    ----------
    m : np.array
        Matriz
    method: int
        Método para o cálculo dos autovetores
    prints: Boolean
        Auxilio em debugging
    Returns
    -------
    numpy.array
        O autovetor principal
    float
        Indice de Inconsistencia por Saaty     
    float
        Percentual de inconsistencia
    '''
    # Etapa 5, via numpy
    # 
    methods = {1: numpy_method,
               2: iteration_method,
               3: mean_normalized_values_method}

    # A funcao de calculo do autovetor retorna os parametros
    max_autovalor, max_autovetor = methods.get(method)(m)
    n = len(m)
    # Etapa 6
    ic = (max_autovalor - n)/(n-1)
    RIss = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.4, 1.45, 1.49]
    RIs = {i: RIss[i-1] for i in range(1, 11)}
    RC = ic/RIs[n]
    max_autovetor = max_autovetor.real/sum(max_autovetor.real)
    if prints:
        print('Autovetor associado ao maior autovalor\n', max_autovetor)
        print('Indice de inconsistencia: ',ic)
        print('Razao de Consistencia:', RC)
    return max_autovetor.real, RC.real, ic.real

In [0]:
def numpy_method(m):
    ''' Encapsulamento numpy '''
    autovalores, autovetores = methods.get(method)(m)
    max_autovetor = autovetores[:,autovalores.argmax()]
    max_autovalor = autovalores.max()
    return max_autovalor, max_autovetor

def iteration_method(A, p=1e-9, max_it=1000):
    '''
    Método para calcular o autovetor principal de uma matriz recíproca através de itereções.   
    
    Parâmetros
    ----------
    A : numpy.array
        Matriz recíproca A n por n
    p : float
        Precisão do Algoritmo
    max_it : int
        N máximo de interações
        
    Retorna
    -------
    numpy.array
        Autovetor principal
    float
        Autovalor máximo      
    
    '''
    
    # Recebe a ordem da matriz
    n = A.shape[0]
    # Inicializar vetor X0
    X0 = np.ones(n)
    # Inicializar variáveis
    Xlast = X0
    λlast = 0 
    λk = 1
    k=0
    
    # Iterar até convergir
    while(abs(λk-λlast)>p or k>max_it):   
        # Guardar último λ
        λlast = λk
        # Fazer kth λk e Xk atualizar
        Yk = A.dot(Xlast)
        λk = np.max(Yk)
        Xk = (1/λk)*Yk
        # Guardar Xk
        Xlast = Xk
        # Incrementar k
        k+=1

    return λk, Xk

def mean_normalized_values_method(A):
    '''
    Método para calcular o principal autovetor de uma matriz recíproca através da média normalizada dos valores.
    
    Parâmetros
    ----------
    A : numpy.array
        Matriz recíproca A n por n

    Returna
    -------
    numpy.array
        Autovetor principal
    float
        Autovalor máximo      
    
    '''
    
    # Cria matriz nula com mesma dimensão de A
    Wi = np.zeros(shape= A.shape)
    
    # Divide cada elemento de A pela somatória dos elementos da sua coluna
    for i in range(len(A)):
        for j in range(len(A[0])):
            sum_col = A[:,j].sum()
            Wi[i][j] = A[i][j]/sum_col
    
    # Cria vetor nulo com tamanho n
    W_vec = np.zeros(shape= (len(Wi),1))
    
    # Soma cada linha e divide por n
    for i in range(len(Wi)):
        W_vec[i] = (Wi[i].sum())/len(Wi[0])

    # Noraliza para obter o autovetor
    W_vec = W_vec/max(W_vec)

    # Calcula o máximo autovalor
    R = A.dot(W_vec)
    sum_R = 0
    for i in range(len(R)):
        sum_R += float(R[i]/W_vec[i])
    lamb_max = (1/len(R))*sum_R
    
    return  lamb_max, W_vec.reshape(1,A.shape[0])[0]

In [0]:
def construir_matriz(nomes, n):
    '''
    nomes: list(string) 
    n: tamanho matriz (n x n)
    '''
    m = np.eye(n)
    for j in range(n):
        for i in range(n):
            if i > j:
                if nomes:
                    nome1, nome2 = nomes[j], nomes[i]
                    val = input('Inserir o peso de {} em relação a {}: '.format(nome1,
                                                                         nome2))
                else:
                    val = input('Inserir o peso do parametro {} em relação a {}: '.format(j+1,
                                                                   i+1))
                # Recebe as relações e verifica se é maior que zero
                val = float(eval(val))
                if val < 0:
                    val = input('Valor negativo, favor reinserir: ')
                    val = float(eval(val))
                m[i,j] = 1/val
                m[j,i] = val

    return m

In [0]:
# Recebe o número de critérios e alternativas do problema analisado:
n_crit = int(input('Qual o numero de criterios desejados? '))
n_alte = int(input('Qual o número de alternativas? '))
yn_crit = input('Deseja inserir o nome dos criterios (s/n)?\n').lower() == 's'
nomes_crit = []
if yn_crit:
    for i in range(n_crit):
        nomes_crit.append(
            input('Insira o nome do {}o criterio: '.format(i+1)))
yn_alte = input('Deseja inserir o nome das alternativas (s/n)? ').lower() == 's'
nomes_alte = []
if yn_alte:
    for i in range(n_alte):
        nomes_alte.append(
            input('Insira o nome da {}a alternativa: '.format(i+1)))

Qual o numero de criterios desejados? 5
Qual o número de alternativas? 3
Deseja inserir o nome dos criterios (s/n)?
s
Insira o nome do 1o criterio: Compl Implementacao
Insira o nome do 2o criterio: Tempo Treinamento
Insira o nome do 3o criterio: Tempo Pred
Insira o nome do 4o criterio: Interpretabilidade
Insira o nome do 5o criterio: Acuracia
Deseja inserir o nome das alternativas (s/n)?
s
Insira o nome da 1a alternativa: SVM
Insira o nome da 2a alternativa: Naive-Bayes
Insira o nome da 3a alternativa: Arvore Decisao


In [0]:
# Construção da matriz de Importância Relativa dos Critérios 
print('Construa a matriz de Importancia Relativa dos Criterios\n')
m_crit = construir_matriz(nomes_crit, n_crit)
print('\nConstrua as matrizes de Comparacoes:\n')

m_comps = []
graus = []
metodo = input('Por favor, escolha o método para cálculo dos autovalores e'
               ' autovetores \n(1: numpy, \n 2: iterativo, \n 3: mean normalized)')
metodo = int(metodo)

Construa a matriz de Importancia Relativa dos Criterios

Inserir o peso de Compl Implementacao em relação a Tempo Treinamento: 0.5
Inserir o peso de Compl Implementacao em relação a Tempo Pred: 0.3333
Inserir o peso de Compl Implementacao em relação a Interpretabilidade: 0.6666
Inserir o peso de Compl Implementacao em relação a Tempo Treino: 0.33333
Inserir o peso de Tempo Treinamento em relação a Tempo Pred: 0.6666
Inserir o peso de Tempo Treinamento em relação a Interpretabilidade: 1.3333
Inserir o peso de Tempo Treinamento em relação a Tempo Treino: 0.6666
Inserir o peso de Tempo Pred em relação a Interpretabilidade: 2
Inserir o peso de Tempo Pred em relação a Tempo Treino: 1
Inserir o peso de Interpretabilidade em relação a Tempo Treino: 0.5

Construa as matrizes de Comparacoes:

Por favor, escolha o método para cálculo dos autovalores e autovetores 
(1: numpy, 
 2: iterativo, 
 3: mean normalized)2


In [0]:
for i in range(n_crit):
    if nomes_crit:
        print('\nMatriz para o critério {}:\n'.format(nomes_crit[i]))
    else:
        print('\nMatriz para o {}o critério:\n'.format(i+1))
    mat = construir_matriz(nomes_alte, n_alte)
    m_comps.append(mat)
    
    # Cálcula medidas de inconsistência para verificação da matriz de importância relativa dos critérios
    res = AHP_graus(mat, metodo)
    graus.append(res)
    print('\nÍndice de Inconsistência proposto por Saaty: {:.2f}%'.format(res[2]*100))
    print('Percentual de inconsistência: {:.2f}%'.format(res[1]*100))
    if res[1] > 0.1:
        print('ATENÇÃO: Percentual de inconsistência > 10%\nVerifique com o especialista')

pesos_crit = AHP_graus(m_crit, 3)[0]
pesos_crit/sum(pesos_crit)
graus_crit = np.array([i[0]/sum(i[0]) for i in graus]).T
print('\nMatriz de graus nos critérios\n',graus_crit)
prioridade = graus_crit.dot(pesos_crit)/sum(graus_crit.dot(pesos_crit))
print('\nNível de Prioridade (0-1):\n')
for i, pri in enumerate(prioridade):
    print('{}\t{:.3f}'.format(nomes_alte[i].capitalize(), pri))


Matriz para o critério Compl Implementacao:

Inserir o peso de SVM em relação a Naive-Bayes: 0.33333
Inserir o peso de SVM em relação a Arvore Decisao: 0.5
Inserir o peso de Naive-Bayes em relação a Arvore Decisao: 1.5

Índice de Inconsistência proposto por Saaty: 0.00%
Percentual de inconsistência: 0.00%

Matriz para o critério Tempo Treinamento:

Inserir o peso de SVM em relação a Naive-Bayes: 0.1666
Inserir o peso de SVM em relação a Arvore Decisao: 0.25
Inserir o peso de Naive-Bayes em relação a Arvore Decisao: 1

Índice de Inconsistência proposto por Saaty: 0.92%
Percentual de inconsistência: 1.76%

Matriz para o critério Tempo Pred:

Inserir o peso de SVM em relação a Naive-Bayes: 1
Inserir o peso de SVM em relação a Arvore Decisao: 1.5
Inserir o peso de Naive-Bayes em relação a Arvore Decisao: 0.66666

Índice de Inconsistência proposto por Saaty: 3.68%
Percentual de inconsistência: 7.07%

Matriz para o critério Interpretabilidade:

Inserir o peso de SVM em relação a Naive-Bayes

# Métodos

## Níveis de Prioridade


|                |  Mean | Iterativo |
|:--------------:|:-----:|:---------:|
|       SVM      | 0.288 |   0.288   |
|   Naive-Bayes  | 0.376 |   0.376   |
| Arvore-Decisao | 0.336 |   0.336   |